In [5]:
!pip install elasticsearch
!pip install --user -U nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 4.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 785.1/785.1 kB 6.4 MB/s eta 0:00:0000:0100:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
from datetime import datetime
from elasticsearch import Elasticsearch

ELASTIC_PASSWORD = "p2iFCHUbC7ze1QoIMVw"

es = Elasticsearch("http://elasticsearch:9200",
                    basic_auth=("elastic", ELASTIC_PASSWORD))

es.info()

ObjectApiResponse({'name': 'es-node', 'cluster_name': 'tdt4117-ir-data-cluster', 'cluster_uuid': '7rryNDgDS1yAM3OPC7QvQA', 'version': {'number': '8.4.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '89f8c6d8429db93b816403ee75e5c270b43a940a', 'build_date': '2022-09-14T16:26:04.382547801Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [32]:
# b) Indexer
import nltk
nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from pathlib import Path
import os
from collections import defaultdict

# Tokenize a document and give out a set without stopwords
def tokenize(document): 
    #Just a list of all stop words
    stop_words = set(stopwords.words('english'))

    # Tokenize or split into a list of words
    word_tokens = word_tokenize(document);
    token_set = set(word_tokens)

    # Adds word to list as lowercase, if not in stop_words
    filtered_sentence = [ w for w in token_set if not w.lower in stop_words ]

    return filtered_sentence


# Take list of documents and gives out tokenized list with the inverted index
def build_inverted_index(documents):
    inverted_index = defaultdict(list)
    # Enumerate gives an index and the value
    for doc_id, document in enumerate(documents):
        tokens = tokenize(document)
        
        for token in tokens: 
            inverted_index[token].append(doc_id)
    return inverted_index

# Writes a file with the terms of the inverted indexes
def save_index(index, filename):
    with open(filename, 'w') as file:
        for term, postings in index.items():
            file.write(f"{term}: {', '.join(map(str, postings))}\n")

def get_content(file_path: Path):
    with file_path.open(mode='r', encoding='utf-8') as file:
        content = file.read()
        return content
                
def get_path():
    current_script_directory = os.path.dirname(os.path.abspath('elk_base_documents_template.ipynb'))
    folder_name = "DataAssignment4"
    path = os.path.join(current_script_directory, folder_name)
    return path

        
def index():
    path = get_path()
    print(path)
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith('.txt'):
                file_path = Path(os.path.join(root, file))
                content = get_content(file_path)

                # Tokenize and invert indexes
                index = build_inverted_index(content)

                # Save the indexes to a file for easy access
                save_index(index, 'output.txt')

                # Return the inverted index
                return index
            
index()

ModuleNotFoundError: No module named 'nltk'

In [12]:
# TODO import documents into elastic
import os
from pathlib import Path

index_name = "a4"

def get_content(file_path: Path):
    with file_path.open(mode='r', encoding='utf-8') as file:
        content = file.read()
        return content

def index_to_elasticsearch(content, doc_id):
    # print(index_name, doc_id, content)
    es.index(index=index_name, id=doc_id, document={'content': content})

def get_path():
    current_script_directory = os.path.dirname(os.path.abspath('elk_base_documents_template.ipynb'))
    folder_name = "DataAssignment4"
    path = os.path.join(current_script_directory, folder_name)
    return path

def add_to_elastic():
    path = get_path()
    print(path)
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith('.txt'):
                file_path = Path(os.path.join(root, file))
                content = get_content(file_path)
                # Use the file name as a unique ID for the document
                doc_id = os.path.splitext(file)[0]
                index_to_elasticsearch(content, doc_id)

add_to_elastic()

/home/jovyan/DataAssignment4


Got 6 Hits:
5.417975
0.1597732
0.15703472
0.15641768
0.15391785
0.14406839


In [9]:
import os
current_script_directory = os.path.dirname(os.path.abspath('elk_base_documents_template.ipynb'))

folder_name = "DataAssignment4"

path = os.path.join(current_script_directory, folder_name)


In [16]:
# TODO Query "claim"
resp = es.get(index=index_name, id="Text1")

es.indices.refresh(index=index_name)

resp = es.search(index=index_name, query={"match": {"content": {"query": "claim"}}})
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit["_score"])

Got 1 Hits:
1.5510131


In [25]:
# TODO Query "claim*"
resp = es.get(index=index_name, id="Text1")

es.indices.refresh(index=index_name)

resp = es.search(index=index_name, query={"match": {"content": "claim*"}})
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit["_score"])

Got 1 Hits:
1.5510131


In [28]:
# TODO Query "claims of duty"
resp = es.get(index=index_name, id="Text1")

es.indices.refresh(index=index_name)

resp = es.search(index=index_name, query={"match": {"content": "claims of duty"}})
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit["_id"],":", hit["_score"])

Got 6 Hits:
Text6 : 5.417975
Text1 : 0.1597732
Text4 : 0.15703472
Text5 : 0.15641768
Text2 : 0.15391785
Text3 : 0.14406839


In [30]:
# TODO Query "claims of duty" in an alternative way
resp = es.get(index=index_name, id="Text1")

es.indices.refresh(index=index_name)

resp = es.search(index=index_name, query={"match": {"content": {"query": "claims of duty"}}})
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit["_id"],":", hit["_score"])

Got 6 Hits:
Text6 : 5.417975
Text1 : 0.1597732
Text4 : 0.15703472
Text5 : 0.15641768
Text2 : 0.15391785
Text3 : 0.14406839
